Author: Ruoyu Liu, Yikai Hao, Yu-Chieh Chen

# Data Structure

## Data Generating Process

We have a source which divides applications into three categories - popular apps, random apps, and malware. For our current model, we extract features, such as package names, invoke type, api name, code block number, app name, class name, and type of apps from the source. These data can help us to determine the relationship between different apps. Also, it helps us to construct the A, B, and P matrix to understand the suspicious api calls to be malware. We format a .py file which is written in Python to get the values we want. (additional information about .py files in Appendix) In order to reduce the space we take up and the run time, we also make some optimizations on code.

## Database

We design a special database to store the data we get. Since our main feature is the api, there are over 2 millions unique apis and more than 50 millions apis appeared in different smali files. In order to save space and time, we use separate csv files to store different unique string values. Instead of using string, we assign an unique id to each string to represent the string value. Then, in the main csv files, we store the unique ids from different references. This will reduce the space and time. The description of our database is appended below.



![database.png](../data/report/database.png)

# EDA

## Statistics

| type | 1 Call API mean | 1 Call API SUM | API | Class | APP
|:------------- |:-------------|:-------------|:-------------|:-------------|:-------------|
|Malware|0.01|159.10|2393.25|406.21|464|
|Popular|0.02|2625.55|26937.55|6123.47|232|
|Random|0.01|1098.03|19719.85|3896.35|232|

464 malwares, 232 popular apps, and 232 random apps are used as source of data generating process. The table shows some statistics we generated from the data. The api column represents the number of unique apis in an app. According to the HinDroid policy, api here means the combination of api and package. The class column talks about the number of classes in an app, which can also be considered as the number of smali files in an app. In addition, we also calculate the apis only being called once in each type and their average among apps. The statistics confirmed our idea that the size of malwares is much smaller than other apps. The number of unique apis other apps have is around 10 times than malware, such difference also exists when we compare the number of classes. What's more, we find out that the apis called once in each type takes up a really small percentage among all api calls.

## Visualization

Some explorations on data are done by using visualization. We plot some graphs about apis, classes, and also PCA. 
 

The graph shows the spread of the number of classes in an app, which can also be understood as the number of smali files in an app. As the graph shows, malwares are condensed together around 0 - 2000. On contrast, random apps and popular apps are more widely spread. In addition, popular apps contain some outliers which have really large numbers. By taking a closer look at the specific number, malwares dominate the range of 0 - 2000. Most of the random apps are in the range 2000 - 7000. Outliers of popular apps are mostly located at the range > 7000.

![class_graph.png](../data/report/class_graph.png)


Then take a look at the number of unique apis. Those graphs look really similar to previous graphs. Malwares still condense at the small number area. Popular apps and random apps are more widely spread over the graph with some outliers. More specifically, malwares often contain 0 - 10000 unique apis. Random apps are mainly located in range 10000 - 25000. Popular apps dominate the range > 25000. Combining the conclusion we get from previous graph and statistics, we reinforce the idea that malwares have a size way smaller than other apps.
![api.png](../data/report/api.png)

We also plot the PCA graph to see if three categories have clear differences in lower dimension. As we can see from the graph, green dots and orange dots are mainly overlapped. It is reasonable since both random apps and popular apps are considered as 'good' apps. For malwares, although part of them are overlapped with other apps, they mostly lie near y = 0 and their x value are always smaller than 0. So there are some features that can be used to split malwares and other apps.

![pca.png](../data/report/pca.png)

# Method

## Baseline

### Summary of Features

The baseline model includes the following features for each app:

| Column Name      | Dtype          |Nominal /Ordinal /Quantitative  | Encoding Method |
|:------------- |:-------------| :-----|:-----|
| number of public methods | `int` | Quantitative | StandardScaler |
| number of protected methods | `int` | Quantitative | StandardScaler |
| number of private methods | `int` | Quantitative | StandardScaler |
| number of code blocks | `int` | Quantitative | StandardScaler |
| number of system apis | `int` | Quantitative | StandardScaler |
| number of string apis| `int` | Quantitative | StandardScaler |
| number of invoke methods | `int` | Quantitative | StandardScaler |
| number of unique apis | `int` | Quantitative | StandardScaler |


### Estimator and Hyperparamter**

We choose three models: Support Vector Machines(SVM), Random Forest Classifier (RF), and Gradient Boosting Classifier (GB) and use `GridSearchCV` to find the optimal hyperparameters for them.

    
| Model | Parameters | 
|:------------- |:-------------|
| SVM | pca__n_components = 0.99, C = 100, gamma =  1, kernel = rbf|
| RF | pca__n_components =  0.99, max_depth =  None, min_samples_split = 3, n_estimators = 50|
| GB | pca__n_components =  0.99, loss = deviance, max_depth = 8, min_samples_split = 7, n_estimators = 100|


### Model Performance

Both **accuracy** and **f1 score** are used as our evaluation method, because in a real android system, there are more benign apps than malware apps. Moreover, both false negatives (malware apps are ignored) and false positives (benign apps are identified as malware) should be eliminated. Therefore, the f1 score may provide more information than accuracy.

| Model | Accuracy (avg)| F1 Score (avg) 
|:------------- |:-------------|:-------------|
|SVM|0.950|0.947|
|RF|0.985|0.985|
|GB|0.981|0.980|

## HinDroid

Hindroid [1] uses a structured heterogeneous information network (HIN) to present the relationship between the apps and APIs. In it, it defines four matrices. Matrix A checks whether an app has a specific API call. Matrix B pairs up the two APIs and checks whether they appeared in the same block of code. Matrix P checks if two APIs have the same package. Matrix I checks if two APIs use the same invoke method. Matrix A and B are used to generate the following kernels to determine malware.  

### Kernel Introduction

- $AA^{T}$

![AA.png | 10x10](../data/report/AA.png)

The meta-path for APP $\xrightarrow[]{\text{contains}}$ API $\xrightarrow[]{\text{$contains^{-1}$}}$ APP. 

The items in $AA^{T}$ represent the number of how many apis are shared between two apps. 


- $ABA^{T}$

![ABA.png | 20x20](../data/report/ABA.png)

The meta-path for APP $\xrightarrow[]{\text{contains}}$ API  $\xrightarrow[]{\text{same code block}}$API$\xrightarrow[]{\text{$contains^{-1}$}}$ APP 

The items in $ABA^{T}$ represent the number of how many apis within the same block are shared between two apps. 



### Kernel EDA

- $AA^{T}$

The following is the statistics of the $AA^{T}$. With 372 malware apps, there is an average of 256.63 common API calls for each two malware apps. With 272 benign apps, there is an average of 4539.17 common API calls existing in the two different benign apps. Between 272 benign and malware apps, there is average of 692.02 common API calls in one malware and one benign apps.


|   | Malware |   Benign  |
|:--------:|:--------:|:------:|
| Malware |  256.63  | 692.02 |
| Benign |  692.02  | 4539.17 |

- $ABA^{T}$


The following is the statistics of the $ABA^{T}$. With 372 malware apps, there is an average of 748.24 APIs within the same block of code that are shared between two malware apps. With 272 benign apps, there is an average of 3260.75 common API calls existing in the two different benign apps sharing in the same block. Between 272 benign and malware apps, there is average of 1546.95 common API calls sharing by the same block in one malware and one benign apps. 



|   | Malware |   Benign  |
|:--------:|:--------:|:------:|
| Malware |  748.24  | 1546,95 |
| Benign |  1546,95  | 3260.75 |

### Model Performance
- $AA^{T}$

|  Dataset | Accuracy |   F-1  | Number of Apps |
|:--------:|:--------:|:------:|:--------------:|
| Training |    1.0   |   1.0  |       744      |
|  Testing |  0.9783  | 0.9787 |       184      |

- $ABA^{T}$

|  Dataset | Accuracy |   F-1  | Number of Apps |
|:--------:|:--------:|:------:|:--------------:|
| Training |  0.8280  | 0.8280 |       744      |
|  Testing |  0.8370  | 0.8454 |       184      |

- **Comparision**

|  Dataset | Accuracy |   F-1  |
|:--------:|:--------:|:------:|
| $AA^{T}$ |  0.9783  | 0.9787 |
|  $ABA^{T}$ |  0.8370  | 0.8454 |

Using the same dataset where training set contains 744 apps and testing set contains 184 apps, $AA^{T}$ outperforms $ABA^{T}$ in both **accuracy** and **f1 score**. 

# Reference

[1]: "HinDroid: An Intelligent Android Malware Detection System Based on Structured Heterogeneous Information Network" https://www.cse.ust.hk/~yqsong/papers/2017-KDD-HINDROID.pdf

# Appendix

- Description of .py file for data generation process
    - util.py 
        - list_flatten (function): It is a function that used to flatten the nested list 
        - unique_id _generator (class): It is a class which assigns an unique id to different values with the same category. It will also save the id and value pair into a csv file.
    - make_dataset.py
        - smali_data (function): It is a function that searches for all the smali files in one application. It takes in a file path which is a path to a certain app and returns a dictionary of smali files' path. 
        - find_app_address (function): It is a function that gets all file paths of apps.
        - stat_lst (function): It is the most important function which helps us extract the features we want. We go into the smali files and extract the features step by step. The most outer information we get is class name and code block. Then, in each class and code block, we search for apis and invoke method's type. We store each type of information in their own array. 
        - stat_df (function): It is a function that combines different arrays we get from stat_lst into a dataframe and then save them into a csv file. It is separated by different categories of app
        - clean_df (function): It is a function that combines some shared features of apps together. For example the class name, api name, etc.


